University of Michigan

Master of Applied Data Science

SIADS699 - Capstone Project

Andre Onofre, Samantha Roska, Sawsan Allam

This Notebook: Grad-CAM for Dermatoscopic Images

Note: the code on this notebook was created with Gemini support

In [ ]:
# Mount Drive
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

In [ ]:
!pip install tf-keras-vis

In [ ]:
# Import Libraries
import tensorflow as tf
import keras
import numpy as np
import pandas as pd
from tensorflow.keras.preprocessing.image import img_to_array
from PIL import Image
from tensorflow.keras.applications.densenet import preprocess_input as densenet_preprocess
from tensorflow.keras.applications.convnext import preprocess_input as convnext_preprocess
from tensorflow.keras.applications.efficientnet import preprocess_input as efficientnet_preprocess
from matplotlib.colors import LinearSegmentedColormap
import matplotlib.pyplot as plt
import cv2
from matplotlib.colors import LinearSegmentedColormap
from tf_keras_vis.gradcam_plus_plus import GradcamPlusPlus
from tf_keras_vis.utils.model_modifiers import ReplaceToLinear
from tf_keras_vis.utils.scores import CategoricalScore

In [ ]:
# Directories
IMAGES_DIR = './Images/BCN20000-all-images-original/bcn_20k_train/'
IMAGES_ARRAY_DIR = './Images_Arrays/'
META_DATA_DIR = './Metadata/'
MODELS_DIR = './Models/'
TEST_IMAGES_DIR = './Images/Images_for_tests/'
TRAINING_RESULTS_DIR = './Training_Results/'

In [ ]:
# Custom Colormap Definition
colors = [
    (0.00, '#4B0082'), # Purple
    (0.25, '#0000FF'), # Blue
    (0.50, '#00FF00'), # Green
    (0.75, '#FFFF00'), # Yellow
    (0.90, '#FF0000'), # Red
    (1.00, '#FFFFFF')  # White
]
CUSTOM_GRAD_CAM_CMAP = LinearSegmentedColormap.from_list('Custom_Purple_to_Red', colors)

In [ ]:
# GradCAM++ Function
def generate_grad_cam_plus_plus(final_model_with_grad_cam: tf.keras.Model,
                                img_tensor: tf.Tensor,
                                display_tensor: tf.Tensor,
                                custom_cmap: LinearSegmentedColormap,
                                last_conv_layer,
                                true_class) -> plt.Figure:

    # Classes Dict
    BCN_classes_dict = {0: 'AK', 1: 'BCC', 2: 'BKL', 3: 'DF', 4: 'MEL',
                        5: 'NV', 6: 'SCC', 7: 'VASC'}

    # Determine the target class
    predictions = final_model_with_grad_cam(img_tensor)
    predictions_np = np.squeeze(predictions[0].numpy()) # Convert tensor to numpy array and remove batch dimension

    # Find the index and probability of the most probable class
    pred_index = np.argmax(predictions_np).item()
    pred_probability = predictions_np[pred_index]

    print(f"Predicted Class: {BCN_classes_dict[pred_index]}")
    print(f"Probability: {pred_probability:.2f}")

    # Print True Class
    print('True Class: ' + true_class)
    print("\n")

    # Initialize GradCAM++ and Score
    gradcam_pp = GradcamPlusPlus(
        final_model_with_grad_cam,
        model_modifier=ReplaceToLinear(),
        clone=True
    )

    # Use the CategoricalScore for the target class index
    score = CategoricalScore([pred_index])

    # Generate the heatmap
    heatmap = gradcam_pp(score,
                         img_tensor,
                         penultimate_layer=last_conv_layer)

    # Prepare for Visualization (Resizing and Coloring)
    heatmap = heatmap[0] # Remove batch dimension (1, H, W) -> (H, W)
    original_img = display_tensor[0].numpy().astype(np.uint8)
    heatmap = cv2.resize(heatmap, (original_img.shape[1], original_img.shape[0]))

    # Apply the custom colormap object to the normalized heatmap
    heatmap_colored = custom_cmap(heatmap)[..., :3] * 255
    heatmap_colored = heatmap_colored.astype(np.uint8)

    # Blend the heatmap with the original image
    overlay = cv2.addWeighted(original_img, 0.7, heatmap_colored, 0.3, 0)

    # Plotting the result
    fig, ax = plt.subplots(1, 2, figsize=(10, 5))

    ax[0].imshow(original_img)
    ax[0].set_title('Original Image')
    ax[0].axis('off')

    ax[1].imshow(overlay)
    ax[1].set_title(f'Grad-CAM++ Overlay (Class: {BCN_classes_dict[pred_index]})')
    ax[1].axis('off')

    return fig

In [ ]:
# Test Function
def test_image_grad_cam(list_images, model_type, final_model_with_grad_cam, last_conv_layer):

  for image_name in list_images:
      image_path = IMAGES_DIR + image_name
      print('\n------------------------------------------------------')
      print(f"\nProcessing image: {image_name}")

      # Load and prepare image array (0-255 RGB)
      image = Image.open(image_path).convert('RGB')
      image_array_writable = img_to_array(image).copy() # Shape (H, W, 3), values 0-255

      # Resize
      inference_image_resized = tf.image.resize(image_array_writable, [112, 112])

      # Add Batch Dimension
      inference_image_array = inference_image_resized.numpy().reshape(1, 112, 112, 3)

      # Create the UNPROCESSED tensor for display
      display_image_tensor = tf.convert_to_tensor(inference_image_array)

      # Execute the normalization based on the CNN type
      if model_type == 'DenseNet121':
        processed_image_array = densenet_preprocess(inference_image_array)

      elif model_type == 'ConvNeXtTiny':
        processed_image_array = convnext_preprocess(inference_image_array)

      elif model_type == 'EfficientNetB2':
        processed_image_array = efficientnet_preprocess(inference_image_array)

      # Convert back to TensorFlow Tensor (THIS IS THE MODEL INPUT)
      inference_image_tensor = tf.convert_to_tensor(processed_image_array)

      # Check True Class
      true_class = df[df['bcn_filename']==image_name]['diagnosis'].item()

      # Generate Grad-CAM++ Figure
      fig = generate_grad_cam_plus_plus(
          final_model_with_grad_cam=final_model_with_grad_cam,
          img_tensor=inference_image_tensor,
          display_tensor=display_image_tensor,
          custom_cmap=CUSTOM_GRAD_CAM_CMAP,
          last_conv_layer=last_conv_layer,
          true_class = true_class
  )

      # Display the figure
      plt.show()

In [ ]:
# Read Dataset
df = pd.read_csv(META_DATA_DIR + 'BCN20000-Metadata-One-Hot.csv')

In [ ]:
# Select N images to test
N = 5
list_images0 = df[df['diagnosis'] == 'AK'].head(N)['bcn_filename'].to_list()
list_images1 = df[df['diagnosis'] == 'BCC'].head(N)['bcn_filename'].to_list()
list_images2 = df[df['diagnosis'] == 'BKL'].head(N)['bcn_filename'].to_list()
list_images3 = df[df['diagnosis'] == 'DF'].head(N)['bcn_filename'].to_list()
list_images4 = df[df['diagnosis'] == 'MEL'].head(N)['bcn_filename'].to_list()
list_images5 = df[df['diagnosis'] == 'NV'].head(N)['bcn_filename'].to_list()
list_images6 = df[df['diagnosis'] == 'SCC'].head(N)['bcn_filename'].to_list()
list_images7 = df[df['diagnosis'] == 'VASC'].head(N)['bcn_filename'].to_list()
list_images_all = list_images0 + list_images1 + list_images2 + list_images3 + list_images4 + \
                  list_images5 + list_images6 + list_images7

In [ ]:
# Load Model and select the layer for GradCAM++
final_model_with_grad_cam = tf.keras.models.load_model(MODELS_DIR + 'BCN20000-V104-DenseNet121.keras')
model_type = 'DenseNet121'
last_conv_layer = 'conv5_block16_concat'
test_image_grad_cam(list_images_all, model_type, final_model_with_grad_cam, last_conv_layer)